# Noisy DNA dataset

Clone the GitHub repository into the folder where this notebook is located:

- GitHub repository (contains the ground truth sequences): https://github.com/MLI-lab/noisy_dna_data_storage

Then, download the noisy reads from the following Figshare link and place them in the data folder of the cloned repository:

- Noisy reads (Figshare): https://figshare.com/s/cd611884b34a8c89f4b4



#### 1.) Load data from file

In [1]:
import os, sys
from pathlib import Path

notebook_dir = Path.cwd()
project_root = (notebook_dir / ".." / "..").resolve()
sys.path.insert(0, str(project_root))  # parent that contains 'src' directory

from src.utils.helper_functions import file_to_list, fastq_to_list


# Path to data directory
data_dir = os.path.join(notebook_dir, 'noisy_dna_data_storage', 'data')

# Full path to the .txt file
orig_file = os.path.join(data_dir, "File1_ODNA.txt")
orig_seqs = file_to_list(orig_file)

# Full path to the .fastq file 
fastq_file = os.path.join(data_dir, "I16_S2_R1_001.fastq.gz")
seqs = fastq_to_list(fastq_file)


#### 2.) Filter out too short/long reads and reads with invalid characters

In [2]:
print(f"sequences are, e.g.,  {seqs[:3]}")
print(f"Ground truth are, e.g.,  {orig_seqs[:3]}")


print("all sequences: ", len(seqs))
print("all orig sequences: ", len(orig_seqs))
# Define valid characters
valid_bases = set("ATCG")

# Filter by length and valid characters
reads = [
    seq for seq in seqs 
    if 55 <= len(seq) <= 70 and set(seq).issubset(valid_bases)
]
print("all trimmed sequences: ",  len(reads))

sequences are, e.g.,  ['GCCTTCACNTGATTAACCCCAGCATGATACTTGTCGAACAACATCACGTGAGGTCAATCTCTCCC', 'TAATCGAGNACCAGAATCGCCCGTCTCCGCATTCACAACAGTCGCAACATTCCCTCC', 'GATGTTAGNTGTCCAATATATACACAACGAGTGTCGCATTCCCCC']
Ground truth are, e.g.,  ['TTTCAGTCCGAATGGGGGGTCCTCGTAGTCTCGCACCTTTTGACAACCCTTCTATCCGGA', 'ATGCCTTCTACAACGCGAGAGGGTAAACGAAGCCGAACTCCCACAACCCTTCTCATGGGT', 'AGCATGGCTGCTCTGCGAGGGGAAGTGCCATTGTCGGTTCGCACAACCCTTCTGTCCTGG']
all sequences:  29303855
all orig sequences:  16383
all trimmed sequences:  14294193


#### 3.) Get index to ground truth sequence map

In [3]:
from collections import Counter
from src.utils.helper_functions import decode_dna_index

start_pos = 42
end_pos = 54
# 24 bit positions is 12 nts, first 5 nts are ACAAC

# Initialize
decoded_binary = []
decoded_ints = []
index_to_dna = {}
gt_map = {}

# Decode each ground truth sequence's index
for orig_seq in orig_seqs:
    idx = orig_seq[start_pos:end_pos]
    if idx.startswith("ACAAC"):
        try:
            binary_str, int_val = decode_dna_index(idx)
            decoded_binary.append(binary_str)
            decoded_ints.append(int_val)
            index_to_dna[int_val] = idx  # maps to DNA index string
            gt_map[int_val] = orig_seq   # maps to full sequence 
        except Exception as e:
            print(f"Failed to decode {idx}: {e}")
    else:
        print(f"Index {idx} does not start with ACAAC.")

duplicates = [k for k, v in Counter(decoded_ints).items() if v > 1]
decoded_set = set(decoded_ints)
expected_set = set(range(16384))
missing = expected_set - decoded_set
extra = decoded_set - expected_set

print("Example mappings:")
for i in range(5):
    print(f"DNA Index: {index_to_dna[decoded_ints[i]]}, Binary = {decoded_binary[i]}, Int = {decoded_ints[i]}")

if missing:
    print(f"Missing {len(missing)} index values. Example(s): {sorted(list(missing))[:10]}")
else:
    print("All integer indices from 0 to 16383 are present.")

if extra:
    print(f"Unexpected extra index values: {sorted(list(extra))[:10]}")
else:
    print("No extra index values found.")

if duplicates:
    print(f"Found {len(duplicates)} duplicated integer indices. Examples: {duplicates[:10]}")
else:
    print("No duplicate integer indices.")


Example mappings:
DNA Index: ACAACCCTTCTA, Binary = 01011111011100, Int = 6108
DNA Index: ACAACCCTTCTC, Binary = 01011111011101, Int = 6109
DNA Index: ACAACCCTTCTG, Binary = 01011111011110, Int = 6110
DNA Index: ACAACCCTTCTT, Binary = 01011111011111, Int = 6111
DNA Index: ACAACCCTTCGA, Binary = 01011111011000, Int = 6104
Missing 1 index values. Example(s): [10275]
No extra index values found.
No duplicate integer indices.


#### 3.) Sweep over sliding windows around true index positions and error thresholds to known index placeholder pattern ( for this dataset true start and end position and threshold 0 worked best)

In [4]:
import os
import csv
import pickle  
from src.utils.helper_functions import cluster_by_index_region


# True index window
start_pos = 42
end_pos = 54
# Tried with max_expand up to 6 and thresholds up to 2 and found that 0 for both wogave lowest miscluster rate (see cell below))
max_expand = 0 # set to larger to perform sweep
thresholds = [0] #[0,1,2] # add 1,2, etc. to allow error threshold

# Configs
print_flag = True

# Run sweep
for i in range(0, max_expand + 1):
    start_window = start_pos - i
    end_window = end_pos + i
    for threshold in thresholds:
        print(f"\nRunning with start={start_window}, end={end_window}, threshold={threshold}")
        clusters, failed = cluster_by_index_region(
            reads=reads,
            decoded_ints=decoded_ints,
            threshold=threshold,
            start_window=start_window,
            end_window=end_window,
            print_flag=print_flag
        )

        # Ensure we are storing sequences, not indices
        for cluster_id, cluster_reads in clusters.items():
            for r in cluster_reads:
                if not isinstance(r, str):
                    raise ValueError(f"Cluster {cluster_id} contains non-string item: {r} (type: {type(r)})")

        # Save CSV version (read strings)
        filename = f"index_clusters_sw{start_window}_ew{end_window}_th{threshold}.csv"
        save_path = os.path.join(data_dir, filename)
        with open(save_path, 'w', newline='') as f:
            writer = csv.writer(f)
            for cluster_reads in clusters.values():
                writer.writerow(cluster_reads)
        print(f"Saved clusters to: {save_path}")

        # Save dictionary version as .pkl
        dict_filename = f"index_clusters_sw{start_window}_ew{end_window}_th{threshold}.pkl"
        dict_path = os.path.join(data_dir, dict_filename)
        with open(dict_path, 'wb') as f:
            pickle.dump(clusters, f)
        print(f"Saved cluster dictionary to: {dict_path}")



Running with start=42, end=54, threshold=0
Processing read 0.00e+00
Processing read 1.00e+06
Processing read 2.00e+06
Processing read 3.00e+06
Processing read 4.00e+06
Processing read 5.00e+06
Processing read 6.00e+06
Processing read 7.00e+06
Processing read 8.00e+06
Processing read 9.00e+06
Processing read 1.00e+07
Processing read 1.10e+07
Processing read 1.20e+07
Processing read 1.30e+07
Processing read 1.40e+07
Number of reads that could not be clustered is 13393118/14294193 (93.70%)
Clustering time taken: 30.43 seconds
Saved clusters to: /workspaces/TReconLM/data/noisy_dna/noisy_dna_data_storage/data/index_clusters_sw42_ew54_th0.csv
Saved cluster dictionary to: /workspaces/TReconLM/data/noisy_dna/noisy_dna_data_storage/data/index_clusters_sw42_ew54_th0.pkl


### 4.) Evaluate sliding‐window and error‐threshold combinations by minimizing misclustering rate


In [5]:
import os
import pickle
import csv
import numpy as np
import matplotlib.pyplot as plt
from Levenshtein import distance as levenshtein_distance

# Configs
expected_mean = 60 * (0.057 + 0.06 + 0.026)  # = 8.58
min_expected = 5
max_expected = 13


results = []

for fname in sorted(os.listdir(data_dir)):
    if not fname.endswith(".pkl"):
        continue

    full_path = os.path.join(data_dir, fname)
    with open(full_path, 'rb') as f:
        clusters = pickle.load(f)  # clusters[int_val] = list of read strings

    distances = []
    filtered_clusters = {}
    clusters_with_outliers = set()
    too_few_edits = 0
    too_many_edits = 0
    total_reads = 0
    misclustered_reads = 0

    for int_val, read_list in clusters.items():
        gt_seq = gt_map.get(int_val)
        if gt_seq is None:
            continue

        cluster_dists = []
        filtered_reads = []

        for read in read_list:

            if not isinstance(read, str) or not isinstance(gt_seq, str):
                print(f"[TYPE ERROR] int_val={int_val} | gt_seq type: {type(gt_seq)} | read type: {type(read)}")
                continue


            dist = levenshtein_distance(gt_seq, read)
            distances.append(dist)
            cluster_dists.append(dist)

            total_reads += 1
            if min_expected <= dist <= max_expected:
                filtered_reads.append(read)
            else:
                misclustered_reads += 1
                clusters_with_outliers.add(int_val)
                if dist < min_expected:
                    too_few_edits += 1
                else:
                    too_many_edits += 1

        if filtered_reads:
            filtered_clusters[int_val] = filtered_reads

    rate = misclustered_reads / total_reads if total_reads > 0 else np.nan
    results.append((fname, total_reads, misclustered_reads, rate))

    print(f"\n{fname:45} | Total: {total_reads:5d} | Misclustered: {misclustered_reads:5d} | Rate: {rate*100:.2f}%")

    # Plot histogram of Levenshtein distances 
    plt.figure(figsize=(8, 5))
    plt.hist(distances, bins=50, edgecolor='black', alpha=0.8)
    plt.axvspan(0, min_expected, color='lightgrey', alpha=0.5, label="Too few edits")
    plt.axvspan(max_expected, max(distances), color='lightgrey', alpha=0.5, label="Too many edits")
    plt.axvline(min_expected, color='red', linestyle='--', label=f"Min Expected ({min_expected})")
    plt.axvline(max_expected, color='red', linestyle='--')
    plt.axvline(expected_mean, color='green', linestyle='--', label=f"Expected Mean ({expected_mean:.2f})")
    plt.xlabel("Levenshtein Distance to Ground Truth")
    plt.ylabel("Frequency")
    plt.title(f"Distance Distribution {fname}")
    plt.legend()
    plt.tight_layout()
    plot_path = os.path.join(data_dir, fname.replace(".pkl", "_dist_hist.pdf"))
    plt.savefig(plot_path)
    plt.close()

    # Plot distribution of filtered cluster sizes 
    cluster_sizes = [len(reads) for reads in filtered_clusters.values()]
    plt.figure(figsize=(8, 5))
    plt.hist(cluster_sizes, bins=50, edgecolor='black', alpha=0.8)
    plt.xlabel("Cluster Size (filtered)")
    plt.ylabel("Frequency")
    plt.title(f"Filtered Cluster Sizes, {fname}")
    plt.tight_layout()
    plot_path = os.path.join(data_dir, fname.replace(".pkl", "_size_hist.pdf"))
    plt.savefig(plot_path)
    plt.close()

# Summary printout
print("\nSorted Summary of All Clusterings")
results.sort(key=lambda x: x[-1])  # sort by miscluster rate
for fname, total, mis, rate in results:
    print(f"{fname:45} | Total: {total:5d} | Misclustered: {mis:5d} | Rate: {rate*100:.2f}%")

# Optional: save summary to CSV
summary_path = os.path.join(data_dir, "clustering_eval_summary.csv")
with open(summary_path, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["filename", "total_reads", "misclustered_reads", "miscluster_rate"])
    for fname, total, mis, rate in results:
        writer.writerow([fname, total, mis, f"{rate:.4f}"])
print(f"\nSaved summary to: {summary_path}")



index_clusters_sw42_ew54_th0.pkl              | Total: 901075 | Misclustered: 685438 | Rate: 76.07%

Sorted Summary of All Clusterings
index_clusters_sw42_ew54_th0.pkl              | Total: 901075 | Misclustered: 685438 | Rate: 76.07%

Saved summary to: /workspaces/TReconLM/data/noisy_dna/noisy_dna_data_storage/data/clustering_eval_summary.csv


### 5.) Split in 80\% train and 10\% validation and test sets.

In [6]:
import os
import pickle
import random
import csv

# Configs
random.seed(42)
pkl_path = os.path.join(data_dir,"index_clusters_sw42_ew54_th0.pkl")

out_dir = os.path.join(data_dir, "splits")
os.makedirs(out_dir, exist_ok=True)
# Load clusters
with open(pkl_path, 'rb') as f:
    clusters = pickle.load(f)


# Filter only valid cluster keys (i.e., those with a ground truth)
valid_ids = [k for k in clusters if k in gt_map]
random.shuffle(valid_ids)

# Split cluster IDs
n = len(valid_ids)
n_train = int(0.8 * n)
n_val = int(0.1 * n)
n_test = n - n_train - n_val

train_ids = valid_ids[:n_train]
val_ids   = valid_ids[n_train:n_train + n_val]
test_ids  = valid_ids[n_train + n_val:]

# Helper to extract clusters and ground truths 
def extract_split_data(split_ids):
    split_clusters = [clusters[k] for k in split_ids]
    split_gts = [gt_map[k] for k in split_ids]
    return split_clusters, split_gts

train_clusters, train_ground_truth = extract_split_data(train_ids)
val_clusters, val_ground_truth     = extract_split_data(val_ids)
test_clusters, test_ground_truth   = extract_split_data(test_ids)

# Optional save read-gt pairs to CSVs
def save_to_csv(clusters, gts, path):
    with open(path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['read', 'ground_truth'])
        for reads, gt in zip(clusters, gts):
            for r in reads:
                writer.writerow([r, gt])

save_to_csv(train_clusters, train_ground_truth, os.path.join(out_dir, "train.csv"))
save_to_csv(val_clusters, val_ground_truth,     os.path.join(out_dir, "val.csv"))
save_to_csv(test_clusters, test_ground_truth,   os.path.join(out_dir, "test.csv"))

print(f"Saved train/val/test splits with {len(train_clusters)} / {len(val_clusters)} / {len(test_clusters)} clusters")


Saved train/val/test splits with 13106 / 1638 / 1639 clusters


### 6.) Filter train and val set to not contain sequences too close to a test set gt sequence

In [7]:
# Filter if too close [5,13] distance to gt in test
import Levenshtein  
from tqdm.notebook import tqdm 

# Define the filter range
min_dist = 5
max_dist = 13

# Create a set of test ground-truth sequences
test_gts = set(test_ground_truth)

def should_remove(read, test_gts):
    """Check if read is too close to any test-set GT"""
    for gt in test_gts:
        dist = Levenshtein.distance(read, gt)
        if min_dist <= dist <= max_dist:
            return True
    return False

def filter_clusters(clusters, gts, test_gts):
    """Remove reads that are too close to any test GT"""
    filtered_clusters = []
    filtered_gts = []
    total_removed = 0
    total_kept = 0

    for reads, gt in zip(clusters, gts):
        new_reads = [r for r in reads if not should_remove(r, test_gts)]
        if new_reads:
            filtered_clusters.append(new_reads)
            filtered_gts.append(gt)
            total_kept += len(new_reads)
            total_removed += len(reads) - len(new_reads)
        else:
            total_removed += len(reads)

    print(f"Removed {total_removed} reads; Kept {total_kept}", flush=True)
    return filtered_clusters, filtered_gts

In [8]:
# Apply to train and val clusters
train_clusters_filtered,   train_gt_filtered   = filter_clusters(train_clusters, train_ground_truth, test_gts)

# Apply to train and val clusters
val_clusters_filtered,   val_gt_filtered   = filter_clusters(val_clusters, val_ground_truth, test_gts)

Removed 30546 reads; Kept 690395
Removed 3905 reads; Kept 87429


In [ ]:
# Remove any clusters that now only have 1 read
def remove_too_small_clusters(clusters, gts, min_size=2, split_name=""):
    total = len(clusters)
    filtered = [(c, gt) for c, gt in zip(clusters, gts) if len(c) >= min_size]
    remaining = len(filtered)
    removed = total - remaining

    print(f"{split_name}: Removed {removed} clusters with fewer than {min_size} reads")
    print(f"{split_name}: {remaining} clusters remain")

    return [c for c, _ in filtered], [gt for _, gt in filtered]

train_clusters_filtered, train_gt_filtered = remove_too_small_clusters(
    train_clusters_filtered, train_gt_filtered, min_size=2, split_name="Train"
)

val_clusters_filtered, val_gt_filtered = remove_too_small_clusters(
    val_clusters_filtered, val_gt_filtered, min_size=2, split_name="Val"
)

n_train_before = len(train_clusters)
n_train_after_filtering = len(train_clusters_filtered)
n_train_fully_removed = n_train_before - n_train_after_filtering

print(f"Train: {n_train_fully_removed} clusters were completely removed after filtering")

# Compute how many clusters were entirely removed after filtering
n_val_before = len(val_clusters)
n_val_after_filtering = len(val_clusters_filtered)
n_val_fully_removed = n_val_before - n_val_after_filtering

print(f"Val: {n_val_fully_removed} clusters were completely removed after filtering")



In [ ]:
# Inspect a few train clusters and their ground truth
for i in range(5):  # show 5 random examples
    print(f"\nCluster {i + 1}:")
    print("Ground truth:", train_ground_truth[i])
    print("Reads:")
    for r in train_clusters[i]:
        print(f"  {r}")

### 7.) Create subclusters of max. 10 reads for test and val set


In [ ]:
import random 
from src.utils.helper_functions import create_subclusters

# Create subclusters for test set. For train and validation set we will dynamically during training sample cluster size. 
train_examples, train_cluster_sizes = create_subclusters(train_clusters_filtered, train_gt_filtered, max_reads=None) 
val_examples, val_cluster_sizes = create_subclusters(val_clusters_filtered, val_gt_filtered)
test_examples, test_cluster_sizes = create_subclusters(test_clusters, test_ground_truth)  

print(f"# Train examples: {len(train_examples)}")
print(f"# Validation examples: {len(val_examples)}")
print(f"# Test examples: {len(test_examples)}")

# Count subclusters with size <= 10 (max cluster size for model context)
train_lte_10 = sum(1 for size in train_cluster_sizes if size <= 10)
val_lte_10 = sum(1 for size in val_cluster_sizes if size <= 10)
test_lte_10 = sum(1 for size in test_cluster_sizes if size <= 10)

print(f"\nSubclusters with size <= 10:")
print(f"Train: {train_lte_10}/{len(train_cluster_sizes)} ({100*train_lte_10/len(train_cluster_sizes):.2f}%)")
print(f"Val:   {val_lte_10}/{len(val_cluster_sizes)} ({100*val_lte_10/len(val_cluster_sizes):.2f}%)")
print(f"Test:  {test_lte_10}/{len(test_cluster_sizes)} ({100*test_lte_10/len(test_cluster_sizes):.2f}%)")


### 8.) Save Train, Val, and Test Data to Disk and Upload Test Data to Weights & Biases

In [ ]:
import os
import sys
import json
import wandb
import torch
import pickle
import numpy as np
from datetime import datetime
from pathlib import Path
from src.data_pkg.prepare import encode_and_pad


# Path Setup

# notebook is in .../TReconLM/data/noisy_dna
project_root = Path().cwd().parents[1]        

sys.path.insert(0, str(project_root))

from src.data_pkg.prepare import encode_list, pad_encoded_data
# Config 
sequence_type = "nuc"
block_size = 1500

# Load vocab
data_pkg_dir = os.path.join(project_root, "src", "data_pkg")
meta_path = os.path.join(data_pkg_dir, f'meta_{sequence_type}.pkl')

if os.path.exists(meta_path):
    with open(meta_path, 'rb') as f:
        meta = pickle.load(f)
    meta_vocab_size = meta['vocab_size']
    stoi, itos = meta['stoi'], meta['itos']
    encode = lambda s: [stoi[c] for c in s]
    decode = lambda l: ''.join([itos[i] for i in l])
else:
    raise FileNotFoundError(f"Meta file not found: {meta_path}")


# Clean test examples by removing trailing C's from reads 
def remove_trailing_Cs(examples):
    cleaned = []
    for ex in examples:
        reads_part, gt = ex.split(":")
        reads = reads_part.split("|")
        reads = [r.rstrip("C") for r in reads]
        cleaned.append("|".join(reads) + ":" + gt)
    return cleaned

test_cleaned_examples = remove_trailing_Cs(test_examples)

# Save raw text data 
def write_data_to_file(filepath, data):
    with open(filepath, 'w') as f:
        for line in data:
            f.write(line.strip() + '\n')

write_data_to_file(os.path.join(out_dir, "train.txt"), train_examples)
write_data_to_file(os.path.join(out_dir, "val.txt"), val_examples)
write_data_to_file(os.path.join(out_dir, "test.txt"), test_examples)
write_data_to_file(os.path.join(out_dir, "test_cleaned.txt"), test_cleaned_examples)

# Generate reads.txt and ground_truth.txt for test set 
def write_reads_and_gt(examples, reads_path, gt_path):
    with open(reads_path, 'w') as rf, open(gt_path, 'w') as gf:
        for ex in examples:
            reads_part, gt = ex.split(":")
            reads = reads_part.split("|")
            for r in reads:
                rf.write(r.strip() + '\n')
            rf.write('===============================\n')
            gf.write(gt.strip() + '\n')

write_reads_and_gt(
    test_examples,
    os.path.join(out_dir, "reads.txt"),
    os.path.join(out_dir, "ground_truth.txt")
)

write_reads_and_gt(
    test_cleaned_examples,
    os.path.join(out_dir, "reads_cleaned.txt"),
    os.path.join(out_dir, "ground_truth_cleaned.txt")
)

# Encode and pad test data (pass all required arguments)
encode_and_pad(test_examples, "test", stoi, block_size, out_dir)
encode_and_pad(test_cleaned_examples, "test_cleaned", stoi, block_size, out_dir)


# Upload to W&B (if under 2 GiB) 
test_files = [
    "test.txt", "test_cleaned.txt",
    "test_x.pt", "test_y.pt",
    "test_cleaned_x.pt", "test_cleaned_y.pt",
    "reads.txt", "ground_truth.txt",
    "reads_cleaned.txt", "ground_truth_cleaned.txt"
]

total_bytes = sum(os.path.getsize(os.path.join(out_dir, f)) for f in test_files)
max_bytes = 2 * 1024**3  # 2 GiB

upload_wandb=False

if total_bytes < max_bytes and upload_wandb:
    wandb_project='TRACE_RECONSTRUCTION'
    wandb_entity='' # Set your W&B entity here
    run = wandb.init(
        project=wandb_project,
        entity=wandb_entity,
        name="file1_test_cleaned_vs_raw",
        job_type="data_processing"
    )

    artifact = wandb.Artifact(
        name=f"File1-CrossTest-test-{datetime.now().strftime('%Y%m%d_%H%M%S')}",
        type="dataset",
        description="Test set with original and cleaned examples",
        metadata={
            "sequence_type": sequence_type,
            "block_size": block_size,
            "vocab_size": meta_vocab_size,
            "includes_cleaned_version": True
        }
    )

    for f in test_files:
        artifact.add_file(os.path.join(out_dir, f))

    run.log_artifact(artifact)
    run.finish()
else:
    print(f"[SKIPPED] Total size {total_bytes / 1024**3:.2f} GiB exceeds W&B limit or upload_wandb set to {upload_wandb}.")

### 9.) Analyze trailing C’s in dataset

In [ ]:
import os
from collections import Counter
import matplotlib.pyplot as plt

def count_trailing_Cs(s):
    count = 0
    for c in reversed(s):
        if c == 'C':
            count += 1
        else:
            break
    return count

def analyze_trailing_Cs(test_path: str):
    trailing_C_lengths = []
    total_reads = 0
    affected_reads = 0

    with open(test_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line or ":" not in line:
                continue
            reads_part, _ = line.split(":")
            reads = reads_part.split("|")

            for r in reads:
                total_reads += 1
                c_tail_len = count_trailing_Cs(r)
                if c_tail_len > 0:
                    trailing_C_lengths.append(c_tail_len)
                    affected_reads += 1

    # Summary 
    print(f"Total reads: {total_reads}")
    print(f"Reads with trailing 'C's: {affected_reads} ({(affected_reads/total_reads)*100:.2f}%)")

    # Histogram 
    if trailing_C_lengths:
        counter = Counter(trailing_C_lengths)
        print("\nTrailing 'C' tail length distribution:")
        for k in sorted(counter):
            print(f"Length {k}: {counter[k]} reads")

        plt.figure(figsize=(6, 4))
        plt.hist(trailing_C_lengths, bins=range(1, max(trailing_C_lengths)+2), align='left', edgecolor='black')
        plt.xlabel("Length of trailing 'C's")
        plt.ylabel("Number of reads")
        plt.title("Distribution of Trailing 'C' Tails in Test Reads")
        plt.grid(True, linestyle='--', linewidth=0.5)
        plt.tight_layout()
        plt.show()
    else:
        print("No trailing 'C's found.")

test_file = os.path.join(out_dir, "test.txt")

analyze_trailing_Cs(test_file)
